In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


pip install -U xlwt

pip install -U openpyxl

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200911


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [39]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202009_ETC")

#input
input_file = "【C057】ETCカード利用状況.xlsx"
input_file2 = "age_code"

In [40]:
#シート名取得
bk = pd.ExcelFile(data_dir/f"{input_file}")
print(bk.sheet_names)

['利用率', '利用・発行状況', '発行者内訳']


ETC集計データ読込（エクセルで吐き出されたファイルを加工する）

In [59]:
df = pd.read_excel(data_dir/f"{input_file}", sheet_name = 2,
                   usecols=['カード種類','本人家族識別','加入年度','年代','性別','ETC利用有無','年間利用件数','年間利用金額'])

In [60]:
#マージするのに余計な文字列を削除
df['年代'] = df['年代'].str.replace('～','-')
df['年代'] = df['年代'].str.replace('歳','')
df['加入年度'] = df['加入年度'].str.replace('年度','')
df['count'] = 1

In [62]:
df['年代'].unique()

array(['35-39', '40-44', '65-69', '45-49', '60-64', '50-54', '70-74',
       '30-34', '25-29', '55-59', '75-79', '80以上', '20-24', '19以下（20未満）'],
      dtype=object)

In [63]:
df.head()

,カード種類,本人家族識別,加入年度,年代,性別,ETC利用有無,年間利用件数,年間利用金額,count
0,VMゴールド,本人,2016,35-39,女性,利用あり,65,61990,1
1,VMゴールド,本人,2018,35-39,女性,利用あり,43,68500,1
2,VMゴールド,本人,2011,40-44,女性,利用あり,6,7280,1
3,VMゴールド,本人,2011,65-69,男性,利用なし,0,0,1
4,VMゴールド,本人,2017,45-49,男性,利用なし,0,0,1


In [64]:
print(df['カード種類'].unique())

['VMゴールド' 'VM一般' 'お得意様ＧＣ' 'ハウスゴールド']


In [65]:
df = df.rename(columns={'カード種類':'card',
                        '本人家族識別':'kazoku',
                        '加入年度':'nendo',
                        '年代':'age',
                        '性別':'gender',
                        'ETC利用有無':'etc_f',
                        '年間利用件数':'annual_etc_trn',
                        '年間利用金額':'annual_etc_amt'})
print(df.dtypes)

card              object
kazoku            object
nendo             object
age               object
gender            object
etc_f             object
annual_etc_trn     int64
annual_etc_amt     int64
count              int64
dtype: object


In [99]:
df['kazoku'].unique()

array(['本人', '家族'], dtype=object)

In [102]:
#ETC
q = """
        SELECT
        card
        ,nendo
        ,age
        ,gender
        ,etc_f
        ,sum(case when kazoku = '本人' then annual_etc_trn end ) as honin_annual_etc_trn
        ,sum(case when kazoku = '本人' then annual_etc_amt end ) as honin_annual_etc_amt
        ,sum(case when kazoku = '家族' then annual_etc_trn end ) as kazoku_annual_etc_trn
        ,sum(case when kazoku = '家族' then annual_etc_amt end ) as kazoku_annual_etc_amt
        ,sum(case when kazoku = '本人' then count end) as honin_etc_count
        ,sum(case when kazoku = '家族' then count end) as kazoku_etc_count
        from 
            df
        group by 
            1,2,3,4,5
        """
df_etc = sqldf(q, locals())
df_etc.head()

,card,nendo,age,gender,etc_f,honin_annual_etc_trn,honin_annual_etc_amt,kazoku_annual_etc_trn,kazoku_annual_etc_amt,honin_etc_count,kazoku_etc_count
0,VMゴールド,2007,35-39,女性,利用なし,0.0,0.0,NaN,NaN,2.0,NaN
1,VMゴールド,2007,35-39,男性,利用あり,109.0,130810.0,NaN,NaN,2.0,NaN
2,VMゴールド,2007,40-44,女性,利用あり,25.0,12920.0,378.0,205510.0,1.0,1.0
3,VMゴールド,2007,40-44,女性,利用なし,0.0,0.0,NaN,NaN,2.0,NaN
4,VMゴールド,2007,40-44,男性,利用あり,451.0,304460.0,261.0,218710.0,8.0,1.0


In [103]:
df_etc['age'].unique()

array(['35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80以上', '30-34', '25-29', '20-24', '19以下（20未満）'],
      dtype=object)

In [104]:
df_etc['age'] = df_etc['age'].str.replace('19以下（20未満）','18-19')#マージするのに余計な文字列を削除

In [105]:
df_etc['age'].unique()

array(['35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80以上', '30-34', '25-29', '20-24', '18-19'],
      dtype=object)

In [106]:
print(df_etc['nendo'].unique())

['2007' '2007以前' '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015'
 '2016' '2017' '2018' '2019']


「所有率」を算出するため、Cognosで別件で集計した口座数データを読込する

In [13]:
input_file = "個人会員残高_from_cognos.xlsx"

In [14]:
df0 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 0)
df1 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 1)
df2 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 2)
df3 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 3)
df4 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 4)
df5 = pd.read_excel(data_dir/f"{input_file}", sheet_name = 5)

In [15]:
df_cog = pd.concat([df0,df1,df2,df3,df4,df5])
df_cog.head()

,会員カード会社コード,カード提携方式コード,カード提携会社コード１＋２,カード種類コード,カードブランドコード,管轄部店コード,加入年度,加入年度区分（Ａ，Ｂ，Ｃ）,職業コード,年齢階層,性別コード,データ基準年月,活口座数,解約口座数
0,155,10,D0001,2,1,0,2006,A,3,7,2,201903,1,0
1,155,10,D0001,2,1,0,2006,A,3,7,2,201904,1,0
2,155,10,D0001,2,1,0,2006,A,3,7,2,201905,1,0
3,155,10,D0001,2,1,0,2006,A,3,7,2,201906,1,0
4,155,10,D0001,2,1,0,2006,A,3,7,2,201907,1,0


In [16]:
df_cog =df_cog[(df_cog["データ基準年月"] == 202002)]
df_cog.shape[0]

19957

In [17]:
df_cog = df_cog.rename(columns={'会員カード会社コード':'kaisha_code',
                        'カード提携方式コード':'houshiki_code',
                        'カード提携会社コード１＋２':'teikeikaisha_code1_2',
                        'カード種類コード':'card_code',
                        'カードブランドコード':'brand_code',
                        '管轄部店コード':'buten_code',
                        '加入年度':'nendo',
                        '加入年度区分（Ａ，Ｂ，Ｃ）':'nendo_kubun',
                        '職業コード':'job_code',
                        '年齢階層':'age_code',
                        '性別コード':'gender_code',
                        'データ基準年月':'cp',
                        '活口座数':'active_account',
                        '解約口座数':'cancel_account'})
print(df_cog.dtypes)

kaisha_code              int64
houshiki_code            int64
teikeikaisha_code1_2    object
card_code                int64
brand_code               int64
buten_code               int64
nendo                    int64
nendo_kubun             object
job_code                object
age_code                 int64
gender_code              int64
cp                       int64
active_account           int64
cancel_account           int64
dtype: object


In [ ]:
#味見

In [49]:
print(df_cog['teikeikaisha_code1_2'].unique())

['D0001' 'D0002' 'D0004' 'D0005' 'D0006' 'D0008' 'D0011' 'D0014' 'D0018'
 'D0021' 'D0022' 'D0023' 'D0024' 'D0050' 'D0101' 'D0102' 'D0104' 'D0105'
 'D0106' 'D0108' 'D0111' 'D0114' 'D0118' 'D0201' 'D0204' 'D0225' 'D0226'
 'D0227' 'D0228' 'D0003' 'D0095']


In [19]:
print(df_cog['card_code'].unique())

[ 2 31  0]


In [20]:
print(df_cog['brand_code'].unique())

[1 2 0]


In [21]:
print(df_cog['buten_code'].unique())

[0]


In [80]:
print(df_cog['nendo'].unique())#年度が細かいのでまとめる必要あり

[2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2005 1976 1980 1982 1988 1989 1991 1992 1994 1997 1998 1999 2000 2001
 2002 2003 2004 1971 1973 1975 1981 1983 1984 1985 1986 1987 1990 1993
 1995 1996 1979 1972 1974 1977 1978]


In [81]:
print(df_cog.dtypes)

kaisha_code              int64
houshiki_code            int64
teikeikaisha_code1_2    object
card_code                int64
brand_code               int64
buten_code               int64
nendo                    int64
nendo_kubun             object
job_code                object
age_code                 int64
gender_code              int64
cp                       int64
active_account           int64
cancel_account           int64
age_range               object
cp_date                 object
bop_date                object
dtype: object


In [23]:
print(df_cog['age_code'].unique())#年代がコードで入っているのでコード表を読み込んで変換

[ 7  5  6  8  9 10 11 12 13 14 15  4  3  2]


In [24]:
age_list = pd.read_csv(data_dir/f"{input_file2}.csv",encoding='shift-jis')
age_list

,age_code,age_range
0,1,0歳-17歳
1,2,18歳-19歳
2,3,20歳-24歳
3,4,25歳-29歳
4,5,30歳-34歳
5,6,35歳-39歳
6,7,40歳-44歳
7,8,45歳-49歳
8,9,50歳-54歳
9,10,55歳-59歳


In [25]:
df_cog = pd.merge(df_cog,age_list, on =['age_code'], how = 'left')

In [87]:
print(df_cog['age_range'].unique())

['40-44' '30-34' '35-39' '45-49' '50-54' '55-59' '60-64' '65-69' '70-74'
 '75-79' '80以上' '25-29' '20-24' '18-19']


In [88]:
df_cog['age_range'] = df_cog['age_range'].str.replace('歳','')

In [89]:
print(df_cog['age_range'].unique())

['40-44' '30-34' '35-39' '45-49' '50-54' '55-59' '60-64' '65-69' '70-74'
 '75-79' '80以上' '25-29' '20-24' '18-19']


In [90]:
print(df_cog['gender_code'].unique())

[2 1 0]


In [91]:
print(df_cog['cp'].unique())

[202002]


In [92]:
_df_cog = df_cog.copy()

In [93]:
#日付データの変換
_df_cog['cp_date'] = pd.to_datetime(_df_cog['cp'], format="%Y%m") + MonthEnd(1)
_df_cog['bop_date'] = pd.to_datetime(_df_cog['cp'], format="%Y%m") + MonthEnd(2)

In [94]:
_df_cog['cp_date'] = _df_cog['cp_date'].dt.strftime('%Y/%m/%d')
_df_cog['bop_date'] = _df_cog['bop_date'].dt.strftime('%Y/%m/%d')

In [95]:
print(_df_cog['cp_date'].unique())

['2020/02/29']


In [96]:
print(_df_cog['bop_date'].unique())

['2020/03/31']


In [97]:
df_cog = _df_cog.copy()

In [82]:
#当月末
q = """
        SELECT
        cp_date
        ,case
            when substr(teikeikaisha_code1_2,1,3) = 'D02' then 'お得意様ＧＣ'
            when substr(teikeikaisha_code1_2,4,5) = '05' then 'ハウスゴールド'
            when card_code = 2 then 'VMゴールド'
            else 'VM一般'
        end as card
        ,age_range as age
        ,case 
            when gender_code = 1 then '男性'
            when gender_code = 2 then '女性'
            else '不明'
        end as gender
        ,case when nendo < 2007 then '2007以前' else nendo end as nendo
        ,sum(active_account) as eop_no_account
        ,sum(cancel_account) as sum_no_cancel
        from 
            df_cog
        where 
            substr(teikeikaisha_code1_2,4,5) not in ('95','50')
        group by
            1,2,3,4,5
        """
df_eop = sqldf(q, locals())
df_eop.head()

,cp_date,card,age,gender,nendo,eop_no_account,sum_no_cancel
0,2020/02/29,VMゴールド,18-19,女性,2018,0,0
1,2020/02/29,VMゴールド,18-19,女性,2019,0,0
2,2020/02/29,VMゴールド,18-19,男性,2018,0,0
3,2020/02/29,VMゴールド,18-19,男性,2019,0,0
4,2020/02/29,VMゴールド,20-24,女性,2013,0,0


In [72]:
print(df_eop['card'].unique())

['VMゴールド' 'VM一般' 'お得意様ＧＣ' 'ハウスゴールド']


In [73]:
print(df_eop['age'].unique())

['18-19' '20-24' '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59'
 '60-64' '65-69' '70-74' '75-79' '80以上']


In [83]:
print(df_eop['nendo'].unique())

[2018 2019 2013 2014 2015 2016 2017 2008 2010 2011 2012 2007 2009 '2007以前']


In [107]:
#マージ
q = """
        SELECT
        t1.*
        ,t2.eop_no_account
        from 
            df_etc t1
        left join 
            df_eop t2
        on t1.card = t2.card
        and t1.nendo = t2.nendo
        and t1.age = t2.age
        and t1.gender = t2.gender
        group by
            1,2,3,4,5
        """
df_etc2 = sqldf(q, locals())
df_etc2

,card,nendo,age,gender,etc_f,honin_annual_etc_trn,honin_annual_etc_amt,kazoku_annual_etc_trn,kazoku_annual_etc_amt,honin_etc_count,kazoku_etc_count,eop_no_account
0,VMゴールド,2007,35-39,女性,利用なし,0.0,0.0,NaN,NaN,2.0,NaN,0.0
1,VMゴールド,2007,35-39,男性,利用あり,109.0,130810.0,NaN,NaN,2.0,NaN,0.0
2,VMゴールド,2007,40-44,女性,利用あり,25.0,12920.0,378.0,205510.0,1.0,1.0,4.0
3,VMゴールド,2007,40-44,女性,利用なし,0.0,0.0,NaN,NaN,2.0,NaN,4.0
4,VMゴールド,2007,40-44,男性,利用あり,451.0,304460.0,261.0,218710.0,8.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1712,ハウスゴールド,2008,65-69,男性,利用あり,3.0,3780.0,NaN,NaN,1.0,NaN,1644.0
1713,ハウスゴールド,2008,70-74,女性,利用あり,24.0,25390.0,NaN,NaN,1.0,NaN,1247.0
1714,ハウスゴールド,2009,65-69,男性,利用なし,0.0,0.0,NaN,NaN,1.0,NaN,5.0
1715,ハウスゴールド,2009,80以上,男性,利用あり,22.0,18170.0,NaN,NaN,1.0,NaN,10.0


In [108]:
#CSV出力
df_etc2.to_csv(data_dir /'集計_etc利用有無別_from_cognos.csv', header=True ,index=None, encoding='shift-jis')